In [1]:
#读取文件中的相关积分的数值并保存
import numpy as np
H_core=np.load("D:\\python\\scf\\1e.npy..bin")
S_overlap=np.load("D:\\python\\scf\\S(1).npy..bin")
double_ele_integral=np.load("D:\\python\\scf\\2e(1).npy..bin")#已经是对称，只用索引就行
dimensions1=H_core.shape
dimensions2=S_overlap.shape
dimensions3=double_ele_integral.shape
print('{},{},{}'.format(dimensions1,dimensions2,dimensions3))



(36, 36),(36, 36),(36, 36, 36, 36)


In [2]:
#对角化矩阵S,得到相关的转换矩阵,计算的前置条件
l,U=np.linalg.eigh(S_overlap)
s_1=np.zeros((36,36))
for i in range(36):
    s_1[i,i]=1.0/l[i]**0.5
X_transfer=np.dot(np.dot(U,s_1),np.linalg.inv(U))
X_adjoint=np.matrix.getH(X_transfer)
P_density=np.zeros((36,36))
print(X_transfer.shape)

(36, 36)


In [3]:

#G矩阵的获得，由密度矩阵对它进行更新，迭代计算是计算密度的更新？
def G_two_elec_integral(density):
    G_matrix=np.zeros(density.shape)
    dim=density.shape[0]
    for u in range(dim):
        for v in range(dim):
            a=0
            for p in range(dim):
                for q in range(dim):
                  a+=density[p,q]*(double_ele_integral[u,v,p,q]-0.5*double_ele_integral[u,q,p,v])
            G_matrix[u,v]=a
    return G_matrix#a的作用是给变量赋值


In [4]:
#得到新的密度矩阵P
def calc_P_rhf(c_new):
    density=np.zeros((36,36))
    for i in range(36):
        for j in range(36):
            for m in range(21):#分子空间波函数的轨道索引
                density[i,j]+=2*c_new[i,m]*c_new[j,m]
    return density

#能量计算
def calc_E0_rhf(density,H_core,Fock):
     H_F=H_core+Fock
     E0=0.
     for u in range(36):
          for v in range(36):
               E0+=density[v,u]*H_F[u,v]
     E0*=0.5
     return E0

In [5]:
for i in range(10):
    G_matrix = G_two_elec_integral(P_density)
    Fock = H_core + G_matrix                                  
    Fock_prime = X_adjoint.dot(Fock.dot(X_transfer))                
    C,C_prime = np.linalg.eigh(Fock_prime)      
    C_new = X_transfer.dot(C_prime)                
    P_density_new = calc_P_rhf(C_new)
    E0 = calc_E0_rhf(P_density, H_core, Fock)
    P_density = P_density_new
    E1 =203.030749104     
    Etot = E0+E1 
 
    print("**** iteration: {} ****".format(i+1))
    print(f"Etot:, {Etot:.9f}")


**** iteration: 1 ****
Etot:, 203.030749104
**** iteration: 2 ****
Etot:, -207.750933094
**** iteration: 3 ****
Etot:, -224.150755368
**** iteration: 4 ****
Etot:, -227.807417291
**** iteration: 5 ****
Etot:, -227.888457290
**** iteration: 6 ****
Etot:, -227.889382874
**** iteration: 7 ****
Etot:, -227.889408633
**** iteration: 8 ****
Etot:, -227.889409839
**** iteration: 9 ****
Etot:, -227.889409914
**** iteration: 10 ****
Etot:, -227.889409922


In [6]:
#得到新的密度矩阵P
density=np.zeros((36,36))
for i in range(36):
    for j in range(36):
        for m in range(21):#分子空间波函数的轨道索引
            density[i,j]+=2*C_new[i,m]*C_new[j,m]
def calc_E0_rhf(density,H_core,Fock,):
    H_F=H_core+Fock
    E0=0.
    for u in range(36):
          for v in range(36):
               E0+=density[v,u]*H_F[u,v]
    E0*=0.5
    return E0
E0=calc_E0_rhf(density,H_core,Fock)+203.030749104     
E0

-227.88940774293218

In [7]:
print(P_density_new)
np.shape(P_density_new)     

[[ 2.07000907e+00 -2.10935005e-01  8.71289817e-04 ... -1.98859201e-03
  -1.71675012e-02  2.71347460e-02]
 [-2.10935005e-01  8.00276885e-01 -1.01837856e-02 ...  7.15829685e-03
   6.57928654e-02 -1.00116517e-01]
 [ 8.71289817e-04 -1.01837856e-02  6.06830210e-01 ...  2.21364631e-03
  -4.16799836e-02  1.07956248e-01]
 ...
 [-1.98859201e-03  7.15829685e-03  2.21364631e-03 ...  6.02988683e-01
  -7.04893081e-02 -3.95532238e-02]
 [-1.71675012e-02  6.57928654e-02 -4.16799836e-02 ... -7.04893081e-02
   6.03036953e-01 -7.04684412e-02]
 [ 2.71347460e-02 -1.00116517e-01  1.07956248e-01 ... -3.95532238e-02
  -7.04684412e-02  6.02988236e-01]]


(36, 36)

In [8]:
print(C_new)
np.shape(C_new)
np.save('xishu3',C_new)

[[-2.78368494e-01  5.79340289e-02 -6.25299389e-01 ... -8.89460322e-02
   2.38493789e-04 -6.37049094e-02]
 [-9.19513961e-03  1.62094497e-03 -1.58210271e-02 ...  6.36015413e-01
  -1.66966434e-03  4.81984916e-01]
 [-4.40651808e-04 -3.09434642e-03  6.06095348e-04 ...  2.16421489e-01
  -6.56841224e-01  3.32577726e-01]
 ...
 [-4.70136820e-03  4.93772502e-04  4.96218668e-04 ...  2.64663819e-01
   1.09755466e-03 -2.64410606e-01]
 [-2.53813648e-03  3.55820035e-03  1.72641365e-03 ...  5.55183371e-02
  -1.08682135e-03  2.64306395e-01]
 [ 6.21567603e-04  2.80143788e-03  3.48347676e-03 ... -2.18434003e-01
   1.39627792e-03 -2.64591486e-01]]


In [9]:
#提取轨道能量并且保存出来
print(C)
np.save('energy',C)

[-11.02931047 -11.02929641 -11.02925699 -11.02894481 -11.0289381
 -11.02879229  -1.08911653  -0.95262022  -0.95257553  -0.76317549
  -0.76312415  -0.65724266  -0.5875972   -0.55242565  -0.53040685
  -0.53038757  -0.45572315  -0.4299568   -0.42986468  -0.27940633
  -0.27933628   0.26924798   0.26933872   0.50428235   0.56473154
   0.63647207   0.63659556   0.70674203   0.72600351   0.72606896
   0.87494982   0.87504104   0.89399909   0.89417432   1.08777639
   1.14436877]


In [10]:

newC_=np.pad(C,(0,4),mode='constant')
b=newC_.reshape((8,5))
print(b)


[[-11.02931047 -11.02929641 -11.02925699 -11.02894481 -11.0289381 ]
 [-11.02879229  -1.08911653  -0.95262022  -0.95257553  -0.76317549]
 [ -0.76312415  -0.65724266  -0.5875972   -0.55242565  -0.53040685]
 [ -0.53038757  -0.45572315  -0.4299568   -0.42986468  -0.27940633]
 [ -0.27933628   0.26924798   0.26933872   0.50428235   0.56473154]
 [  0.63647207   0.63659556   0.70674203   0.72600351   0.72606896]
 [  0.87494982   0.87504104   0.89399909   0.89417432   1.08777639]
 [  1.14436877   0.           0.           0.           0.        ]]


In [11]:
import pandas as pd
df=pd.DataFrame(b)
print(df)


           0          1          2          3          4
0 -11.029310 -11.029296 -11.029257 -11.028945 -11.028938
1 -11.028792  -1.089117  -0.952620  -0.952576  -0.763175
2  -0.763124  -0.657243  -0.587597  -0.552426  -0.530407
3  -0.530388  -0.455723  -0.429957  -0.429865  -0.279406
4  -0.279336   0.269248   0.269339   0.504282   0.564732
5   0.636472   0.636596   0.706742   0.726004   0.726069
6   0.874950   0.875041   0.893999   0.894174   1.087776
7   1.144369   0.000000   0.000000   0.000000   0.000000
